# 线性回归的简洁实现

## 通过使用深度学习框架，简洁地实现【线性回归模型】生成数据集

In [2]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b,1000)

## 调用框架中现有的API来读取数据

In [3]:
def load_array(data_array, batch_size, is_train=True):
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_array)    # ＊号在python 语言中调用函数时参数前使用表示解包
                                                 # 列表前加星号是把列表元素分别当做参数穿进去
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[ 0.5379, -0.7015],
         [ 1.0574, -0.2538],
         [ 0.4616,  0.7537],
         [-0.5723,  0.1642],
         [ 0.1123,  0.4096],
         [-0.8071, -2.8755],
         [-0.6924, -0.5492],
         [ 0.1746, -0.7372],
         [-0.6308,  0.4593],
         [ 0.4383, -0.1025]]),
 tensor([[ 7.6695],
         [ 7.1779],
         [ 2.5575],
         [ 2.4946],
         [ 3.0228],
         [12.3703],
         [ 4.6708],
         [ 7.0584],
         [ 1.3677],
         [ 5.4206]])]

## 使用框架的预定义好的层

In [6]:
# 【nn】是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2,1))
# Sequential一个有序的容器，神经网络模块将按照在传入构造器的顺序依次被添加到计算图中执行， #  同时以神经网络模块为元素的有序字典也可以作为传入参数。

## 初始化模型参数

In [7]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

## 计算均方误差使用的是【MSELoss】类，也称平方L2 范数

In [9]:
loss = nn.MSELoss()

## 实例化【SGD】实例

In [11]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

## 训练过程代码与从0开始实现时所作的非常相似

In [12]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)    # net 本身含有w和b，所以不需要再传入了。loss里面已经求和了
        trainer.zero_grad()
        l.backward()
        trainer.step()    # 更新模型
    l = loss(net(features), labels)    # 扫完一遍数据之后，把所有的features放到net里面，与所有的labels计算一次loss，然后打印出来
    print(f'epoch {epoch + 1}, loss{l:f}')

epoch 1, loss0.000314
epoch 2, loss0.000099
epoch 3, loss0.000099
